In [ ]:
import os
import io
import matlab.engine
import sys
import copy
sys.path.append("../")
import od.vars as GV
import od.engine

import time
start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
print(matlab.engine.find_matlab())
print(os.getcwd())
MATLAB_ENG = matlab.engine.connect_matlab()
od.engine.MATLAB_ENG = MATLAB_ENG
MATLAB_ENG.addpath(os.getcwd() + "/../matlab/")
MATLAB_ENG.addpath(os.getcwd() + "/../matlab/SelectCQI_bySNR/")
MATLAB_ENG.addpath(os.getcwd() + "/../matlab/PlannerV1/")
print("Done")

In [ ]:
from numpy import random

In [ ]:
max_group_per_qos = 10
# QoS_GP_CONF = [
#     [
#         {
#             'gid': 0,
#             'qos': 0.0,
#             'rbf_w': 2.0,
#             'rbf_h': 1.0,
#             'sinr_max': 30.0,
#             'pwr_req_dBm': 22.996,
#             'pwr_ext_dBm': -7.3592430462207306,
#             'rem_bits': 123456789,
#             'mem_num': 8.0,
#             'eager_rate': 3.445574916313482
#         }
#     ],
#     [
#         {
#             'gid': float(i + 1),
#             'qos': 1.0,
#             **random.choice([{'rbf_w': 2.0, 'rbf_h': 1.0}, {'rbf_w': 1.0, 'rbf_h': 2.0}]),
#             'sinr_max': float(random.randint(150, 300) / 10),
#             **random.choice([
#                 {'pwr_req_dBm': 22.956, 'pwr_ext_dBm': 3.034702226027002},
#                 {'pwr_req_dBm': 22.959, 'pwr_ext_dBm': 2.729511581869497},
#                 {'pwr_req_dBm': 22.955, 'pwr_ext_dBm': 3.1318014527925526}
#             ]),
#             'rem_bits': float(random.randint(1000, 5000)),
#             'mem_num': float(random.randint(3, 10)),
#             'eager_rate': float(random.randint(1000, 5000) / 10)
#         }
#         for i in range(max_group_per_qos)
#     ] 
# ]
QoS_GP_CONF = [
    [
        {
            'gid': 0,
            'qos': 0.0,
            'rbf_w': 2.0,
            'rbf_h': 1.0,
            'sinr_max': float(random.randint(10, 200) / 10),
            **random.choice([
                {'pwr_req_dBm': 22.956, 'pwr_ext_dBm': 3.034702226027002},
                {'pwr_req_dBm': 22.959, 'pwr_ext_dBm': 2.729511581869497},
                {'pwr_req_dBm': 22.955, 'pwr_ext_dBm': 3.1318014527925526},
                {'pwr_req_dBm': 22.996, 'pwr_ext_dBm': -7.3592430462207306,}
            ]),
            
            'rem_bits': 123456789,
            'mem_num': 8.0,
            'eager_rate': 3.445574916313482
        }
    ],
    [
        {
            'gid': float(i + 1),
            'qos': 1.0,
            'rbf_w': 2.0,
            'rbf_h': 1.0,
            'sinr_max': float(random.randint(10, 200) / 10),
            **random.choice([
                {'pwr_req_dBm': 22.956, 'pwr_ext_dBm': 3.034702226027002},
                {'pwr_req_dBm': 22.959, 'pwr_ext_dBm': 2.729511581869497},
                {'pwr_req_dBm': 22.955, 'pwr_ext_dBm': 3.1318014527925526}
            ]),
            'rem_bits': float(random.randint(1000, 50000)),
            'mem_num': float(random.randint(3, 10)),
            'eager_rate': float(random.randint(1000, 10000) / 10)
        }
        for i in range(max_group_per_qos)
    ] 
]
# RES_CONF = {'rbf_h': float(random.randint(1,5)*10) , 'rbf_w': 2.0, 'max_pwr_dBm': 23.0}
# QoS_GP_CONF = [
#     [
#         {
#             'gid': 5.0,
#             'qos': 1.0,
#             'rbf_w': 2.0,
#             'rbf_h': 1.0,
#             'sinr_max': 30.0,
#             'pwr_req_dBm': 22.996,
#             'pwr_ext_dBm': -7.3592430462207306,
#             'rem_bits': 4125123412,
#             'mem_num': 8.0,
#             'eager_rate': 3.445574916313482
#         },
#         {
#             'gid': 6.0,
#             'qos': 1.0,
#             'rbf_w': 1.0,
#             'rbf_h': 2.0,
#             'sinr_max': 30.0,
#             'pwr_req_dBm': 22.996,
#             'pwr_ext_dBm': -7.3592430462207306,
#             'rem_bits': 40978.0,
#             'mem_num': 8.0,
#             'eager_rate': 4.445574916313482
#         },
#         {
#             'gid': 7.0,
#             'qos': 1.0,
#             'rbf_w': 2.0,
#             'rbf_h': 1.0,
#             'sinr_max': 30.0,
#             'pwr_req_dBm': 22.996,
#             'pwr_ext_dBm': -7.3592430462207306,
#             'rem_bits': 12314413.0,
#             'mem_num': 8.0,
#             'eager_rate': 5.445574916313482
#         }
#     ]
# ]
# RES_CONF = {'rbf_h': 6 , 'rbf_w': 2.0, 'max_pwr_dBm': 23.0}
# QoS_GP_CONF = [[
#     {'gid': 1.0, 'qos': 1.0, 'rbf_w': 1.0, 'rbf_h': 2.0, 'sinr_max': 27.0, 'pwr_req_dBm': 22.955, 'pwr_ext_dBm': 3.1318014527925526, 'rem_bits': 46984.0, 'mem_num': 3.0, 'eager_rate': 2397.1}, 
#     {'gid': 2.0, 'qos': 1.0, 'rbf_w': 2.0, 'rbf_h': 1.0, 'sinr_max': 19.0, 'pwr_req_dBm': 22.955, 'pwr_ext_dBm': 3.1318014527925526, 'rem_bits': 20810.0, 'mem_num': 8.0, 'eager_rate': 1533.1}, 
#     {'gid': 3.0, 'qos': 1.0, 'rbf_w': 1.0, 'rbf_h': 2.0, 'sinr_max': 21.3, 'pwr_req_dBm': 22.956, 'pwr_ext_dBm': 3.034702226027002, 'rem_bits': 33628.0, 'mem_num': 9.0, 'eager_rate': 1295.4}]]
# {'rbf_h': 20.0, 'rbf_w': 2.0, 'max_pwr_dBm': 23.0}
# RES_CONF = {'rbf_h': 20.0, 'rbf_w': 2.0, 'max_pwr_dBm': 23.0}
# QoS_GP_CONF = [[{'gid': 0.0, 'qos': 0.0, 'rbf_w': 2.0, 'rbf_h': 1.0, 'sinr_max': 42.0489549035036, 'pwr_req_dBm': 22.954, 'pwr_ext_dBm': 3.2267555049165395, 'rem_bits': 8944.0, 'mem_num': 54.0, 'eager_rate': 21.806089326894988}], [{'gid': 3.0, 'qos': 1.0, 'rbf_w': 2.0, 'rbf_h': 1.0, 'sinr_max': 35.44738430728282, 'pwr_req_dBm': 22.955, 'pwr_ext_dBm': 3.1318014527925526, 'rem_bits': 37248.0, 'mem_num': 10.0, 'eager_rate': 2.7873663144293546}, {'gid': 4.0, 'qos': 1.0, 'rbf_w': 2.0, 'rbf_h': 1.0, 'sinr_max': 43.746509887342896, 'pwr_req_dBm': 22.954, 'pwr_ext_dBm': 3.2267555049165395, 'rem_bits': 10384.0, 'mem_num': 11.0, 'eager_rate': 10.471926131566418}, {'gid': 2.0, 'qos': 1.0, 'rbf_w': 2.0, 'rbf_h': 1.0, 'sinr_max': 27.52880883218017, 'pwr_req_dBm': 22.962, 'pwr_ext_dBm': 2.4010067070409153, 'rem_bits': 80760.0, 'mem_num': 11.0, 'eager_rate': 1.613756837887903}, {'gid': 5.0, 'qos': 1.0, 'rbf_w': 2.0, 'rbf_h': 1.0, 'sinr_max': 43.91098432147112, 'pwr_req_dBm': 22.954, 'pwr_ext_dBm': 3.2267555049165395, 'rem_bits': 29424.0, 'mem_num': 8.0, 'eager_rate': 2.7459461699705714}, {'gid': 6.0, 'qos': 1.0, 'rbf_w': 2.0, 'rbf_h': 1.0, 'sinr_max': 30.183178573993388, 'pwr_req_dBm': 22.958, 'pwr_ext_dBm': 2.833666714960843, 'rem_bits': 3824.0, 'mem_num': 8.0, 'eager_rate': 12.799092369933515}, {'gid': 8.0, 'qos': 1.0, 'rbf_w': 2.0, 'rbf_h': 1.0, 'sinr_max': 30.387825608684604, 'pwr_req_dBm': 22.958, 'pwr_ext_dBm': 2.833666714960843, 'rem_bits': 11416.0, 'mem_num': 4.0, 'eager_rate': 2.221565157629364}, {'gid': 7.0, 'qos': 1.0, 'rbf_w': 2.0, 'rbf_h': 1.0, 'sinr_max': 46.35760479174594, 'pwr_req_dBm': 22.954, 'pwr_ext_dBm': 3.2267555049165395, 'rem_bits': 13040.0, 'mem_num': 2.0, 'eager_rate': 0.40392450795598345}]]
RES_CONF = {'rbf_h': 50.0, 'rbf_w': 2.0, 'max_pwr_dBm': 23.0}
print(QoS_GP_CONF)
print(RES_CONF)


In [ ]:
def GetTotalSubframeCapacity(alloc_report):
    total = 0
    for ts in alloc_report.values():
        for ts_rb in ts.values():
            for cqi, num in ts_rb.items():
                cqi = int(cqi[1:])
                total+=(MATLAB_ENG.GetThroughputPerRB(cqi,14) * num)
    return total

In [53]:
def Testify(QoS_GP_CONF,RES_CONF):
    ##################################
    out = io.StringIO()
    try:
        alloc_report, exitflag = MATLAB_ENG.PlannerV1(
                    RES_CONF, QoS_GP_CONF, nargout=2, stdout=out
        )
        # alloc_report, exitflag = MATLAB_ENG.PlannerV1(RES_CONF, QoS_GP_CONF, nargout=2, stdout=out)
    except Exception as e:
        raise e
    # for gid in sorted(alloc_report.keys()):
    #     print(gid,alloc_report[gid])
    optim = GetTotalSubframeCapacity(alloc_report)
    ##################################
    allocator = ResourceAllocatorNomaApprox(copy.deepcopy(RES_CONF),copy.deepcopy(QoS_GP_CONF))
    alloc_report = allocator()
    # for gid in sorted(alloc_report.keys()):
    #     print(gid,alloc_report[gid])
    approx = GetTotalSubframeCapacity(alloc_report)
    ##################################
    print("Ratio:",approx/optim)
QoS_GP_CONF[0][0]['rem_bits'] =20000
while(QoS_GP_CONF[0][0]['rem_bits'] > 100):
    QoS_GP_CONF[0][0]['rem_bits'] =round(QoS_GP_CONF[0][0]['rem_bits'] * 0.8)
    print(QoS_GP_CONF[0][0]['rem_bits'])
    Testify(QoS_GP_CONF,RES_CONF)

16000
Allocate For:
{'gid': 0, 'qos': 0.0, 'rbf_w': 2.0, 'rbf_h': 1.0, 'sinr_max': 4.3, 'pwr_req_dBm': 22.956, 'pwr_ext_dBm': 3.034702226027002, 'rem_bits': 16000, 'mem_num': 8.0, 'eager_rate': 3.445574916313482}
{'max_sinr_noise_reciprocal': 0.013626991969948698, 'max_cqi': 6, 'max_sinr_req_pwr_mW': 197.5149622060758, 'max_sinr_ext_pwr_mW': 2.0112692908120664, 'min_sinr_req_pwr_mW': 14.983042106226586}
{'gid': 0, 'qos': 0.0, 'rbf_w': 2.0, 'rbf_h': 1.0, 'sinr_max': 4.3, 'pwr_req_dBm': 22.956, 'pwr_ext_dBm': 3.034702226027002, 'rem_bits': 15802, 'mem_num': 8.0, 'eager_rate': 3.445574916313482} [((g0:c6),rem_pwr:2.0112692908120664)] {'max_sinr_noise_reciprocal': 0.013626991969948698, 'max_cqi': 6, 'max_sinr_req_pwr_mW': 197.5149622060758, 'max_sinr_ext_pwr_mW': 2.0112692908120664, 'min_sinr_req_pwr_mW': 14.983042106226586}
{'gid': 0, 'qos': 0.0, 'rbf_w': 2.0, 'rbf_h': 1.0, 'sinr_max': 4.3, 'pwr_req_dBm': 22.956, 'pwr_ext_dBm': 3.034702226027002, 'rem_bits': 15604, 'mem_num': 8.0, 'eager_

In [ ]:
QoS_GP_CONF

In [54]:
QoS_GP_CONF[0][0]['rem_bits'] = 2000

In [62]:
out = io.StringIO()
try:
    future = MATLAB_ENG.PlannerV1(
                RES_CONF, QoS_GP_CONF, nargout=2, stdout=out, background=True
    )
    alloc_report, exitflag = future.result(timeout=60)
    # alloc_report, exitflag = MATLAB_ENG.PlannerV1(RES_CONF, QoS_GP_CONF, nargout=2, stdout=out)
except Exception as e:
    raise e
for gid in sorted(alloc_report.keys()):
    print(gid,alloc_report[gid])
print(GetTotalSubframeCapacity(alloc_report))

TimeoutError: timeout from execution of the MATLAB function

In [65]:
import importlib
importlib.reload(od.network.allocator)
from od.network.allocator import ResourceAllocatorNomaApprox

In [66]:
allocator = ResourceAllocatorNomaApprox(copy.deepcopy(RES_CONF),copy.deepcopy(QoS_GP_CONF))
alloc_report = allocator()
for gid in sorted(alloc_report.keys()):
    print(gid,alloc_report[gid])
GetTotalSubframeCapacity(alloc_report)

[(0, 0.04695282171646047)]
Allocate For:
{'gid': 0, 'qos': 0.0, 'rbf_w': 2.0, 'rbf_h': 1.0, 'sinr_max': 4.3, 'pwr_req_dBm': 22.956, 'pwr_ext_dBm': 3.034702226027002, 'rem_bits': 2000, 'mem_num': 8.0, 'eager_rate': 3.445574916313482}
{'max_sinr_noise_reciprocal': 0.013626991969948698, 'max_cqi': 6, 'max_sinr_req_pwr_mW': 197.5149622060758, 'max_sinr_ext_pwr_mW': 2.0112692908120664, 'min_sinr_req_pwr_mW': 14.983042106226586}
{'gid': 0, 'qos': 0.0, 'rbf_w': 2.0, 'rbf_h': 1.0, 'sinr_max': 4.3, 'pwr_req_dBm': 22.956, 'pwr_ext_dBm': 3.034702226027002, 'rem_bits': 1802, 'mem_num': 8.0, 'eager_rate': 3.445574916313482} [((g0:c6),rem_pwr:2.0112692908120664)] {'max_sinr_noise_reciprocal': 0.013626991969948698, 'max_cqi': 6, 'max_sinr_req_pwr_mW': 197.5149622060758, 'max_sinr_ext_pwr_mW': 2.0112692908120664, 'min_sinr_req_pwr_mW': 14.983042106226586}
{'gid': 0, 'qos': 0.0, 'rbf_w': 2.0, 'rbf_h': 1.0, 'sinr_max': 4.3, 'pwr_req_dBm': 22.956, 'pwr_ext_dBm': 3.034702226027002, 'rem_bits': 1604, 'mem_

26212

In [58]:
for cqi in range(1,16):
    print(cqi,MATLAB_ENG.GetThroughputPerRB(cqi,14))

1 26
2 39
3 63
4 101
5 147
6 198
7 248
8 322
9 404
10 459
11 558
12 656
13 760
14 859
15 933


In [ ]:
print(allocator.spare_inner_rb)
print(allocator.solid_normal_rb)
print(allocator.spare_hybrid_rb)
print(allocator.solid_hybrid_rb)

In [ ]:
import math
_SINR = 10**(3.1318014527925526/10) * (10**(19.0/10)/ 10**(23/10))
SINR = 10 * math.log10(_SINR)
print(_SINR,SINR)
MATLAB_ENG.SelectCQI_BLER10P(SINR)

In [ ]:
for conf in QoS_GP_CONF[0]:
    print(MATLAB_ENG.SelectCQI_BLER10P(conf["sinr_max"]) * conf["eager_rate"])